In [1]:
# 程序结构设计
#1. 从东方财富网获取股票列表
#2. 根据股票列表逐个到百度股票获取个股信息
#3. 将结果存储到文件

In [61]:
import requests
from bs4 import BeautifulSoup
import re
import traceback

def getHTMLText(url, code='utf-8'):
    try:
        
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = code
        return r.text
    except:
        print('爬取失败')

def getStockList(lst, stockURL):
    html = getHTMLText(stockURL, 'GB2312')
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.find_all('a')
    for i in a:
        try:
            href = i.attrs['href']
            lst.append(re.findall(r"[s][hz]\d{6}", href)[0]) ## 匹配 a 标签中 href 属性以 s 开头，中间是 h 或 z ，最后是6位数字
        except:
            continue

def getStockInfo(lst, stockURL, fpath):
    ## 去掉列表里的重复选项--将列表转换为集合再转换为列表
    lst = list(set(lst))
    count = 0
    for stock in lst:
        url = stockURL + stock[-6:]
        html = getHTMLText(url)
        try:
            if html == '': ## 判断是否空页面
                continue
            infoDict = {} ## 定义一个字典，存储股票信息
            soup = BeautifulSoup(html, 'html.parser')
            stockInfo = soup.find('div', attrs={'class':'stock-info'})

            name = stockInfo.find_all(attrs={'class':'name'})[0]
            price = stockInfo.find_all(attrs={'class': 'latest'})[0]
            infoDict.update({'股票名称':name.text.split()[0], '最新行情':price.text.split()[0]})


            keyList = stockInfo.find_all('dt')
            valueList = stockInfo.find_all('dd')

            for i in range(len(keyList)):
                key = keyList[i].text
                val = valueList[i].text
                infoDict[key] = val


            ## 将字典写入文件中
            with open(fpath, 'a', encoding='utf-8') as f:
                f.write(str(infoDict) + '\n')
                count = count + 1
                ## 增加动态进度显示
                print('\r当前进度：{:.2f}%'.format(count*100/len(lst)), end='')

        except:
            traceback.print_exc()  ## 获得发生异常的错误信息
            continue

def main():
    stock_list_url = 'http://app.finance.ifeng.com/list/stock.php?t=ha'
    stock_info_url = 'https://www.laohu8.com/stock/'
    output_file = 'D:\\BaiduStockInfo.txt'
    slist = []
    getStockList(slist, stock_list_url)
    getStockInfo(slist, stock_info_url, output_file)

main()

当前进度：100.00%

In [28]:
#以上课堂代码，由于网页变动，已经无法获取了   改成上面的

当前进度：100.00%